In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 구글 드라이브 파일 다운로드
# !tar -zxvf /content/drive/MyDrive/ROKEY/data.tar.gz -C /content/
!unzip /content/drive/MyDrive/ROKEY/pdata.zip -d /content/

Archive:  /content/drive/MyDrive/ROKEY/pdata.zip
   creating: /content/pdata/
  inflating: /content/pdata/1733290405.9079654.jpg  
  inflating: /content/pdata/1733290408.752.jpg  
  inflating: /content/pdata/1733290439.1920936.txt  
  inflating: /content/pdata/1733290457.2158823.txt  
  inflating: /content/pdata/1733290396.1357458.jpg  
  inflating: /content/pdata/1733290385.199751.txt  
  inflating: /content/pdata/1733290400.8200512.txt  
  inflating: /content/pdata/1733290469.1534083.txt  
  inflating: /content/pdata/1733290407.9113054.jpg  
  inflating: /content/pdata/1733290458.2159545.txt  
  inflating: /content/pdata/1733290460.780423.jpg  
  inflating: /content/pdata/1733290459.215886.txt  
  inflating: /content/pdata/1733290416.360022.txt  
  inflating: /content/pdata/1733290465.6281986.txt  
  inflating: /content/pdata/1733290434.904286.txt  
  inflating: /content/pdata/1733290466.3481872.jpg  
  inflating: /content/pdata/1733290388.2438698.txt  
  inflating: /content/pdata/17

In [3]:
%cd pdata

/content/pdata


In [4]:
classes = ["corn\n", "cat"]
with open("/content/vocnames.txt", 'w') as f:
      f.writelines(classes)

# manifest.txt 추가하기
마지막 줄에 빈 문자열 있는지 확인

In [5]:
import os
import shutil
from tqdm import tqdm

# 데이터셋 폴더 경로
data_root = "/content/split_data"

# val, train 데이터 디렉토리 정의
val_root = os.path.join(data_root, "val") # /content/split_data/val
train_root = os.path.join(data_root, "train") # /content/split_data/train

# test 데이터 디렉토리 정의
test_root = os.path.join(data_root, "test") # /content/split_data/test

# val, train 디렉토리 없으면 생성
os.makedirs(val_root, exist_ok=True)
os.makedirs(train_root, exist_ok=True)
os.makedirs(test_root, exist_ok=True)

# 이미지 파일 경로 목록이 저장된 파일 열기
with open("/content/manifest.txt") as f:
    files = f.readlines() # 경로를 리스트로 읽어들임

# 파일 처리
for idx, img_path in tqdm(enumerate(files)):
    img_src = img_path.split('\n')[0]
    # print(img_src)
    txt_src = os.path.splitext(img_src)[0] + ".txt"
    img_name = os.path.split(img_src)[-1]
    text_name = os.path.split(txt_src)[-1]

    # 경로 설정
    # 데이터셋의 5%를 테스트 데이터셋으로 사용  7
    if idx % 20 < 1:
        img_dst = os.path.join(test_root, img_name) # 이미지 파일 경로
        text_dst = os.path.join(test_root, text_name) # 텍스트 파일 경로

    # 데이터의 15%를 검증 데이터셋으로 사용 21
    elif idx % 20 < 4:
        img_dst = os.path.join(val_root, img_name)
        text_dst = os.path.join(val_root, text_name)

    # 나머지 80%는 훈련 데이터셋으로 사용 134-28=106
    else:
        img_dst = os.path.join(train_root, img_name)
        text_dst = os.path.join(train_root, text_name)
    # print(img_src, img_dst)
    # print(os.path.exists(img_src), os.path.exists(img_dst))
    shutil.copy2(img_src, img_dst)
    shutil.copy2(txt_src, text_dst)


339it [00:00, 4033.48it/s]


In [6]:
import yaml

data = {'train' : '/content/split_data/train',
        'test': '/content/split_data/test',
        'val' : '/content/split_data/val',
        'nc': 2,
        'names': ["corn", "cat"]
}

with open("/content/split_data/custom_data.yaml", 'w') as f:
  yaml.dump(data, f)

In [7]:
# yaml 파일 확인
with open("/content/split_data/custom_data.yaml", 'r') as f:
  view_yaml = yaml.safe_load(f)
  display(view_yaml)

{'names': ['corn', 'cat'],
 'nc': 2,
 'test': '/content/split_data/test',
 'train': '/content/split_data/train',
 'val': '/content/split_data/val'}

In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 23.2 MB/s eta 0:00:00


In [9]:
import ultralytics

ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


In [10]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt") # load a pretrained model

100%|██████████| 6.25M/6.25M [00:00<00:00, 108MB/s]


In [ ]:
print(type(model.names),len(model.names))

print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [ ]:
# 캐시 지우기
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Train the model
# model.train(data='/content/split_data/custom_data.yaml', epochs= 50, patience = 5, batch=32)
# model.train(data='/content/split_data/custom_data.yaml', epochs= 1, patience = 1, batch=32)
model.train(data='/content/split_data/custom_data.yaml', epochs= 200, patience = 30, batch=32)

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/split_data/custom_data.yaml, epochs=200, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train532, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning /content/split_data/train.cache... 271 images, 0 backgrounds, 0 corrupt: 100%|██████████| 271/271 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/split_data/val.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]


Plotting labels to runs/detect/train532/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train532
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         51         51      0.998          1      0.995      0.988
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 60, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



90 epochs completed in 0.175 hours.
Optimizer stripped from runs/detect/train532/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train532/weights/best.pt, 6.2MB

Validating runs/detect/train532/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


                   all         51         51      0.979      0.978      0.995      0.989
                  corn         21         21          1      0.956      0.995      0.995
                   cat         30         30      0.958          1      0.995      0.984
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train532


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a50fa8b1ea0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
print(type(model.names),len(model.names))

print(model.names)

<class 'dict'> 2
{0: 'corn', 1: 'cat'}


# 예측

In [ ]:
# 테스트 데이터셋으로 잘된지 예측
# 학습된 이미지 출력이 있는 폴더 경로
results = model.predict(source ='/content/split_data/test/', save=True)


image 1/17 /content/split_data/test/1733290399.8198094.jpg: 480x640 1 corn, 12.5ms
image 2/17 /content/split_data/test/1733290408.752.jpg: 480x640 1 corn, 10.1ms
image 3/17 /content/split_data/test/1733290439.1920936.jpg: 480x640 1 corn, 6.7ms
image 4/17 /content/split_data/test/1733290467.0279307.jpg: 480x640 1 corn, 6.3ms
image 5/17 /content/split_data/test/1733290470.391911.jpg: 480x640 1 corn, 6.4ms
image 6/17 /content/split_data/test/1733290490.9401548.jpg: 480x640 1 corn, 6.3ms
image 7/17 /content/split_data/test/1733290514.8122017.jpg: 480x640 1 corn, 6.6ms
image 8/17 /content/split_data/test/1733290857.688662.jpg: 480x640 1 cat, 6.5ms
image 9/17 /content/split_data/test/1733290861.5732372.jpg: 480x640 1 cat, 6.5ms
image 10/17 /content/split_data/test/1733290877.7934222.jpg: 480x640 (no detections), 6.4ms
image 11/17 /content/split_data/test/1733290894.8971386.jpg: 480x640 1 cat, 6.5ms
image 12/17 /content/split_data/test/1733290910.4783092.jpg: 480x640 1 cat, 6.3ms
image 13/17

In [ ]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'p1', 1: 'p2'}
obb: None
orig_img: array([[[197, 180, 167],
        [200, 183, 170],
        [201, 185, 169],
        ...,
        [178, 161, 158],
        [176, 161, 158],
        [177, 162, 159]],

       [[194, 178, 166],
        [197, 181, 169],
        [198, 182, 169],
        ...,
        [176, 161, 158],
        [175, 160, 157],
        [176, 161, 158]],

       [[189, 179, 169],
        [191, 181, 171],
        [194, 182, 172],
        ...,
        [176, 161, 159],
        [175, 160, 158],
        [173, 160, 158]],

       ...,

       [[ 68,  16,   4],
        [ 66,  12,   1],
        [ 71,  16,   3],
        ...,
        [ 44,   0,   0],
        [ 44,   0,   0],
        [ 45,   1,   0]],

       [[ 68,  15,   2],
        [ 67,  14,   1],
        [ 72,  17,   4],
        ...,
        [ 46,   0,   0],
        [ 45,   0,   0],
       

# boxes 좌표값
형식: x_center, y_center, width, height

좌표 단위: 정규화 (0~1 범위)

In [ ]:
for result in results[0]:
    boxes = result.boxes.cpu().numpy()                                # get boxes on cpu in numpy
    print('class : ', boxes.cls[0])
    print('name : ', results[0].names[int(boxes.cls[0])])
    print('confidence :', boxes.conf[0])
    print('xywhn : ', boxes.xywhn[0])
    print(boxes.xywhn[0][0], boxes.xywhn[0][1], boxes.xywhn[0][2], boxes.xywhn[0][3])

class :  0.0
name :  corn
confidence : 0.8235717
xywhn :  [    0.47279     0.47076     0.30644     0.76631]
0.47279492 0.47076118 0.30644435 0.76631445


In [ ]:
for i in range(len(results)):
  for result in results[i]:
      boxes = result.boxes.cpu().numpy()                                # get boxes on cpu in numpy
      print('class : ', boxes.cls[0])
      print('name : ', results[0].names[int(boxes.cls[0])])
      print('confidence :', boxes.conf[0])
      print('xywhn : ', boxes.xywhn[0])
      print(boxes.xywhn[0][0], boxes.xywhn[0][1], boxes.xywhn[0][2], boxes.xywhn[0][3])

class :  0.0
name :  corn
confidence : 0.8235717
xywhn :  [    0.47279     0.47076     0.30644     0.76631]
0.47279492 0.47076118 0.30644435 0.76631445
class :  0.0
name :  corn
confidence : 0.76765335
xywhn :  [    0.16713     0.46428     0.30223      0.7323]
0.16713476 0.4642793 0.30222997 0.73229754
class :  0.0
name :  corn
confidence : 0.9083518
xywhn :  [    0.67343     0.45501     0.38406     0.75208]
0.67343426 0.4550059 0.3840618 0.75208
class :  0.0
name :  corn
confidence : 0.82752043
xywhn :  [     0.5823      0.4634     0.32389     0.78418]
0.5823029 0.46339935 0.32388657 0.7841823
class :  0.0
name :  corn
confidence : 0.87905407
xywhn :  [    0.48315     0.46847     0.32869     0.78447]
0.48314905 0.46846643 0.32868975 0.78447366
class :  0.0
name :  corn
confidence : 0.82870257
xywhn :  [    0.73283     0.43199      0.3097     0.67086]
0.73283434 0.43199196 0.30969524 0.6708586
class :  0.0
name :  corn
confidence : 0.8747233
xywhn :  [    0.67678     0.46963      0.370

# 예측 결과 bbox로 확인

In [ ]:
import glob
import random
import cv2
from google.colab.patches import cv2_imshow
# 테스트 데이터셋 (위에서 저장된) 경로 가져오기
image_paths = glob.glob('/content/pdata/runs/detect/train5322/*.jpg')

# 이미지가 없는 경우 에러 처리
if not image_paths:
    print("이미지 파일이 없습니다.")
else:
    # 테스트 이미지 예측 결과 모두 확인
    for path in image_paths:
      img = cv2.imread(path)
      cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

# best.pt 저장

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%cp runs/detect/train6/weights/best.pt /content/drive/MyDrive/ROKEY
# %cp runs/detect/train532/weights/last.pt /content/drive/MyDrive/ROKEY

# 모델 이어서 학습

In [11]:
# 모델 이어서 학습
from ultralytics import YOLO

# model = YOLO("/content/drive/MyDrive/ROKEY/last.pt")
model = YOLO("/content/drive/MyDrive/ROKEY/best.pt")

# Resume training
# model.train(data='/content/split_data/custom_data.yaml', epochs= 200, batch=32, resume=True)
model.train(data='/content/split_data/custom_data.yaml', epochs=200, batch=32, pretrained='/content/drive/MyDrive/ROKEY/best.pt')

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/ROKEY/best.pt, data=/content/split_data/custom_data.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=/content/drive/MyDrive/ROKEY/best.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

100%|██████████| 755k/755k [00:00<00:00, 20.9MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 5.35M/5.35M [00:00<00:00, 110MB/s]


AMP: checks passed ✅


...

                   all         51         51      0.999          1      0.995      0.987
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



103 epochs completed in 0.164 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.11it/s]


                   all         51         51      0.998          1      0.995       0.99
                  corn         21         21      0.997          1      0.995      0.995
                   cat         30         30      0.998          1      0.995      0.986
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a2771674940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [12]:
results = model.predict(source ='/content/split_data/test/', save=True)


image 1/17 /content/split_data/test/1733290399.8198094.jpg: 480x640 1 corn, 1 cat, 62.8ms
image 2/17 /content/split_data/test/1733290408.752.jpg: 480x640 1 corn, 9.1ms
image 3/17 /content/split_data/test/1733290439.1920936.jpg: 480x640 1 corn, 8.4ms
image 4/17 /content/split_data/test/1733290467.0279307.jpg: 480x640 1 corn, 8.0ms
image 5/17 /content/split_data/test/1733290470.391911.jpg: 480x640 1 corn, 7.5ms
image 6/17 /content/split_data/test/1733290490.9401548.jpg: 480x640 1 corn, 7.5ms
image 7/17 /content/split_data/test/1733290514.8122017.jpg: 480x640 1 corn, 7.6ms
image 8/17 /content/split_data/test/1733290857.688662.jpg: 480x640 1 cat, 10.2ms
image 9/17 /content/split_data/test/1733290861.5732372.jpg: 480x640 1 cat, 10.4ms
image 10/17 /content/split_data/test/1733290877.7934222.jpg: 480x640 1 cat, 9.3ms
image 11/17 /content/split_data/test/1733290894.8971386.jpg: 480x640 1 cat, 8.8ms
image 12/17 /content/split_data/test/1733290910.4783092.jpg: 480x640 1 cat, 8.9ms
image 13/17 /

In [13]:
import glob
import random
import cv2
from google.colab.patches import cv2_imshow

# 테스트 데이터셋 (위에서 저장된) 경로 가져오기
image_paths = glob.glob('/content/pdata/runs/detect/train2/*.jpg')

# 이미지가 없는 경우 에러 처리
if not image_paths:
    print("이미지 파일이 없습니다.")
else:
    # 테스트 이미지 예측 결과 모두 확인
    for path in image_paths:
      img = cv2.imread(path)
      cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [15]:
%cp runs/detect/train/weights/best.pt /content/drive/MyDrive/ROKEY